In [1]:
import robosuite 
import dexmimicgen  

import h5py 
import imageio 
import numpy as np
import time 
import os
import json

'''

1. Loading dual robot environment
'''

# def get_env_metadata_from_dataset(dataset_path, ds_format="robomimic"):
#     """
#     Retrieves env metadata from dataset.

#     Args:
#         dataset_path (str): path to dataset

#     Returns:
#         env_meta (dict): environment metadata. Contains 3 keys:

#             :`'env_name'`: name of environment
#             :`'type'`: type of environment, should be a value in EB.EnvType
#             :`'env_kwargs'`: dictionary of keyword arguments to pass to environment constructor
#     """
#     dataset_path = os.path.expanduser(dataset_path)
#     f = h5py.File(dataset_path, "r")
#     if ds_format == "robomimic":
#         env_meta = json.loads(f["data"].attrs["env_args"])
#     else:
#         raise ValueError
#     f.close()
#     return env_meta

from vishal_dev.utils import get_env_metadata_from_dataset

dataset_path = "/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/dexmimicgen/datasets/two_arm_box_cleanup.hdf5"

env_meta = get_env_metadata_from_dataset(dataset_path)
print(env_meta)

write_to_video = False

env_kwargs = env_meta['env_kwargs']
env_kwargs["env_name"] = env_meta["env_name"]
env_kwargs["has_renderer"] = True
env_kwargs["renderer"] = "mjviewer"
env_kwargs["has_offscreen_renderer"] = write_to_video # False # write_video
env_kwargs["use_camera_obs"] = False

env_kwargs.pop("env_lang")

env = robosuite.make(**env_kwargs)

[robosuite WARNING] No private macro file found! (macros.py:57)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:58)
[robosuite WARNING] To setup, run: python /home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/robosuite/robosuite/scripts/setup_macros.py (macros.py:59)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


{'env_name': 'TwoArmBoxCleanup', 'env_version': '1.5.1', 'type': 1, 'env_kwargs': {'robots': ['PandaDexRH', 'PandaDexLH'], 'controller_configs': {'type': 'BASIC', 'body_parts': {'right': {'type': 'OSC_POSE', 'input_max': 1, 'input_min': -1, 'output_max': [0.05, 0.05, 0.05, 0.5, 0.5, 0.5], 'output_min': [-0.05, -0.05, -0.05, -0.5, -0.5, -0.5], 'kp': 150, 'damping_ratio': 1, 'impedance_mode': 'fixed', 'kp_limits': [0, 300], 'damping_ratio_limits': [0, 10], 'position_limits': None, 'orientation_limits': None, 'uncouple_pos_ori': True, 'input_type': 'delta', 'input_ref_frame': 'base', 'interpolation': None, 'ramp_ratio': 0.2, 'gripper': {'type': 'GRIP', 'use_action_scaling': False}}}}, 'translucent_robot': False, 'env_configuration': 'parallel', 'reward_shaping': False, 'camera_names': ['agentview', 'robot0_eye_in_hand', 'robot1_eye_in_hand'], 'camera_heights': 84, 'camera_widths': 84, 'has_renderer': False, 'has_offscreen_renderer': True, 'ignore_done': True, 'use_object_obs': True, 'use_

In [2]:
import pose_utils_vishal_from_mimicgen as PoseUtils
import robosuite.utils.transform_utils as T

In [3]:
'''
Extracting a demonstration from the dataset
'''

data = h5py.File(dataset_path, "r")
print(f"Keys in the dataset: {data['data'].keys()}")
print(f"Number of demonstrations in the dataset: {len(data['data'].keys())}")
demo_id = 'demo_15'
print(f"Length of demo {demo_id}: {data['data'][demo_id]['obs']['robot0_eef_pos'].shape[0]}")
print(f"Observations shape: {data['data'][demo_id]['obs'].keys()}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Actions: {data['data'][demo_id]['actions']}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Action at step 0: {data['data'][demo_id]['actions'][0]}")

action_trajectory = data['data'][demo_id]['actions']
obs_trajectory = data['data'][demo_id]['obs']
states_trajectory = data['data'][demo_id]['states']

eef_poses = data['data'][demo_id]['datagen_info']['eef_pose']
gripper_actions = data['data'][demo_id]['datagen_info']['gripper_action']
target_eef_poses = data['data'][demo_id]['datagen_info']['target_pose']

Keys in the dataset: <KeysViewHDF5 ['demo_0', 'demo_1', 'demo_10', 'demo_100', 'demo_1000', 'demo_1001', 'demo_1002', 'demo_1003', 'demo_1004', 'demo_1005', 'demo_1006', 'demo_1007', 'demo_1008', 'demo_1009', 'demo_101', 'demo_1010', 'demo_1011', 'demo_1012', 'demo_1013', 'demo_1014', 'demo_1015', 'demo_102', 'demo_103', 'demo_104', 'demo_105', 'demo_106', 'demo_107', 'demo_108', 'demo_109', 'demo_11', 'demo_110', 'demo_111', 'demo_112', 'demo_113', 'demo_114', 'demo_115', 'demo_116', 'demo_117', 'demo_118', 'demo_119', 'demo_12', 'demo_120', 'demo_121', 'demo_122', 'demo_123', 'demo_124', 'demo_125', 'demo_126', 'demo_127', 'demo_128', 'demo_129', 'demo_13', 'demo_130', 'demo_131', 'demo_132', 'demo_133', 'demo_134', 'demo_135', 'demo_136', 'demo_137', 'demo_138', 'demo_139', 'demo_14', 'demo_140', 'demo_141', 'demo_142', 'demo_143', 'demo_144', 'demo_145', 'demo_146', 'demo_147', 'demo_148', 'demo_149', 'demo_15', 'demo_150', 'demo_151', 'demo_152', 'demo_153', 'demo_154', 'demo_155'

In [4]:
from vishal_dev.utils import just_reset, reset_to_state, reset_to

In [5]:
ep = demo_id
states = data["data/{}/states".format(ep)][()]
initial_state = dict(states=states[0])
initial_state["model"] = data["data/{}".format(ep)].attrs["model_file"]
# if args.use_current_model:
initial_state["model"] = env.sim.model.get_xml()
initial_state["ep_meta"] = data["data/{}".format(ep)].attrs.get("ep_meta", None)

In [6]:
'''
Replay a demonstration
'''
# reset_to_state(env, states_trajectory[0])
# just_reset(env, states_trajectory[25])

for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i]
    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)

In [7]:
max_dpos = 0.05
max_drot = 0.5

from vishal_dev.utils import eef_to_action_single_arm, eef_to_action

In [8]:
'''
Data class for source and transformed segments

inputs:
    arm1_src_traj: (N, 4, 4) numpy array
    arm2_src_traj: (N, 4, 4) numpy array
    starting_lid_pose: (4, 4) numpy array
    starting_box_pose: (4, 4) numpy array

to_maintain:
    arm1_src_segment1: (N, 4, 4) numpy array
    arm1_src_segment2: (N, 4, 4) numpy array
    arm2_src_segment1: (N, 4, 4) numpy array
    arm2_src_segment2: (N, 4, 4) numpy array
    starting_lid_pose: (4, 4) numpy array
    starting_box_pose: (4, 4) numpy array
    arm1_tgt_segment1: (N, 4, 4) numpy array
    arm1_tgt_segment2: (N, 4, 4) numpy array
    arm2_tgt_segment1: (N, 4, 4) numpy array
    arm2_tgt_segment2: (N, 4, 4) numpy array
    
    INTERPOLATED TRAJECTORIES
    arm1_interp_toseg1: (N, 4, 4) numpy array
    arm1_interp_toseg2: (N, 4, 4) numpy array
    arm2_interp_toseg1: (N, 4, 4) numpy array
    arm2_interp_toseg2: (N, 4, 4) numpy array
'''

'\nData class for source and transformed segments\n\ninputs:\n    arm1_src_traj: (N, 4, 4) numpy array\n    arm2_src_traj: (N, 4, 4) numpy array\n    starting_lid_pose: (4, 4) numpy array\n    starting_box_pose: (4, 4) numpy array\n\nto_maintain:\n    arm1_src_segment1: (N, 4, 4) numpy array\n    arm1_src_segment2: (N, 4, 4) numpy array\n    arm2_src_segment1: (N, 4, 4) numpy array\n    arm2_src_segment2: (N, 4, 4) numpy array\n    starting_lid_pose: (4, 4) numpy array\n    starting_box_pose: (4, 4) numpy array\n    arm1_tgt_segment1: (N, 4, 4) numpy array\n    arm1_tgt_segment2: (N, 4, 4) numpy array\n    arm2_tgt_segment1: (N, 4, 4) numpy array\n    arm2_tgt_segment2: (N, 4, 4) numpy array\n    \n    INTERPOLATED TRAJECTORIES\n    arm1_interp_toseg1: (N, 4, 4) numpy array\n    arm1_interp_toseg2: (N, 4, 4) numpy array\n    arm2_interp_toseg1: (N, 4, 4) numpy array\n    arm2_interp_toseg2: (N, 4, 4) numpy array\n'

In [9]:
# obfull[0].keys()

In [10]:
'''
Now prepare the data
'''
obfull = just_reset(env, states_trajectory[0])
src_lid_pos = data['data'][demo_id]['datagen_info']['object_poses']['lid'][0][:3, 3]
src_lid_mat = data['data'][demo_id]['datagen_info']['object_poses']['lid'][0][:3, :3].reshape(3, 3)
src_box_pos = data['data'][demo_id]['datagen_info']['object_poses']['box'][0][:3, 3]
src_box_mat = data['data'][demo_id]['datagen_info']['object_poses']['box'][0][:3, :3].reshape(3, 3)
src_lid_pose = PoseUtils.make_pose(src_lid_pos, src_lid_mat)
src_box_pose = PoseUtils.make_pose(src_box_pos, src_box_mat)
print("Starting lid pose: ", src_lid_pose)
print("Starting box pose: ", src_box_pose)

new_lid_pos = env.sim.data.body('lid_obj_root').xpos
new_lid_mat = env.sim.data.body('lid_obj_root').xmat.reshape(3, 3)
new_box_pos = env.sim.data.body('box_obj_root').xpos
new_box_mat = env.sim.data.body('box_obj_root').xmat.reshape(3, 3)
new_lid_pose = PoseUtils.make_pose(new_lid_pos, new_lid_mat)
new_box_pose = PoseUtils.make_pose(new_box_pos, new_box_mat)
print("New lid pose: ", new_lid_pose)
print("New box pose: ", new_box_pose)

cur_arm1_eef_pos = obfull[0]['robot0_eef_pos']
cur_arm1_eef_mat = obfull[0]['robot0_eef_quat']
cur_arm1_eef_mat = T.quat2mat(cur_arm1_eef_mat)
cur_arm1_eef_pose = PoseUtils.make_pose(cur_arm1_eef_pos, cur_arm1_eef_mat)
print("Current arm1 eef pose: ", cur_arm1_eef_pose)
cur_arm2_eef_pos = obfull[0]['robot1_eef_pos']
cur_arm2_eef_mat = obfull[0]['robot1_eef_quat']
cur_arm2_eef_mat = T.quat2mat(cur_arm2_eef_mat)
cur_arm2_eef_pose = PoseUtils.make_pose(cur_arm2_eef_pos, cur_arm2_eef_mat)
print("Current arm2 eef pose: ", cur_arm2_eef_pose)
cur_eef_poses = np.zeros((2, 4, 4))
cur_eef_poses[0, :, :] = cur_arm1_eef_pose
cur_eef_poses[1, :, :] = cur_arm2_eef_pose

Starting lid pose:  [[ 1.          0.          0.         -0.00800182]
 [ 0.          1.          0.          0.2       ]
 [ 0.          0.          1.          0.725     ]
 [ 0.          0.          0.          1.        ]]
Starting box pose:  [[ 1.          0.          0.         -0.00938105]
 [ 0.          1.          0.         -0.15      ]
 [ 0.          0.          1.          0.776     ]
 [ 0.          0.          0.          1.        ]]
New lid pose:  [[ 1.          0.          0.         -0.03174408]
 [ 0.          1.          0.          0.2       ]
 [ 0.          0.          1.          0.72476748]
 [ 0.          0.          0.          1.        ]]
New box pose:  [[ 1.00000000e+00  9.40871506e-25 -4.55314588e-14  2.94228664e-02]
 [-1.01275739e-24  1.00000000e+00 -1.57881787e-12 -1.50000000e-01]
 [ 4.55314588e-14  1.57881787e-12  1.00000000e+00  7.74126732e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Current arm1 eef pose:  [[ 0.99324838  0.01639

In [11]:
'''
Transforming the source trajectories to the new object positions
'''

'''
Transforming two segments one by one

Process:
1. Transform segment 1 using object pose at start of segment 1
2. Interpolate between starting pose to starting pose of segment 1
3. Transform segment 2 using object pose at start of segment 2
4. Interpolate between end of segment 1 to starting pose of segment 2

'''

from pose_utils_vishal_from_mimicgen import transform_source_data_segment_using_object_pose
from pose_utils_vishal_from_mimicgen import interpolate_poses
from pose_utils_vishal_from_mimicgen import make_pose
from pose_utils_vishal_from_mimicgen import unmake_pose

def transform_two_segments_for_one_arm(
        segment1,
        segment2,
        src_obj1_pose,
        cur_obj1_pose,
        src_obj2_pose,
        cur_obj2_pose,
        starting_eef_pose,
    ):
    '''Transform 2 segments of trajectory from source to current based on object poses

    Args:
        segment1 (list): list of eef_poses for segment 1 (T, 4, 4)
        segment2 (list): list of eef_poses for segment 2 (T, 4, 4)
        src_obj1_pose (np.ndarray): source object pose (4, 4)
        cur_obj1_pose (np.ndarray): current object pose (4, 4)
        src_obj2_pose (np.ndarray): source object pose (4, 4)
        cur_obj2_pose (np.ndarray): current object pose (4, 4)
        starting_eef_pose (list): starting eef pose (4, 4)
    Returns:
        
    '''
    # Transform segments into their corresponding target object poses
    transformed_segment1 = transform_source_data_segment_using_object_pose(
        src_eef_poses=segment1,
        src_obj_pose=src_obj1_pose,
        obj_pose=cur_obj1_pose,
    )
    transformed_segment2 = transform_source_data_segment_using_object_pose(
        src_eef_poses=segment2,
        src_obj_pose=src_obj2_pose,
        obj_pose=cur_obj2_pose,
    )

    # Interpolate between starting pose to starting pose of segment 1
    interp_poses0, num_steps = interpolated_poses = interpolate_poses(
        pose_1=starting_eef_pose,
        pose_2=transformed_segment1[0],
        num_steps=10,
        # step_size=0.1, # one of step size or num_steps is needed
    )

    # Interpolate between end of segment 1 to starting pose of segment 2
    interp_poses1, num_steps = interpolated_poses = interpolate_poses(
        pose_1=transformed_segment1[-1],
        pose_2=transformed_segment2[0],
        num_steps=10,
        # step_size=0.1, # one of step size or num_steps is needed
    )

    # Create a dict of all trajectories 
    overall_trajectory = {
        'interp_to_segment1': interp_poses0,
        'segment1': transformed_segment1,
        'interp_to_segment2': interp_poses1,
        'segment2': transformed_segment2,
    }

    return overall_trajectory

def get_transformed_segments_for_both_arms(
        segment1,
        segment2,
        src_obj1_pose,
        cur_obj1_pose,
        src_obj2_pose,
        cur_obj2_pose,
        starting_eef_pose,
    ):
    '''

    args:
        segment1 (np.ndarray): (N, 2, 4, 4)
        segment2 (np.ndarray): (M, 2, 4, 4) 
        src_obj1_pose (np.ndarray): (4, 4)
        cur_obj1_pose (np.ndarray): (4, 4)
        src_obj2_pose (np.ndarray): (4, 4)
        cur_obj2_pose (np.ndarray): (4, 4)
        starting_eef_pose (list): list of two eef poses [(4, 4), (4, 4)]
    returns:
        dict: {
            "arm1": {
                "overall_trajectory": overall_trajectory_arm1 (dict): {
                    'interp_to_segment1': interp_poses0,
                    'segment1': transformed_segment1,
                    'interp_to_segment2': interp_poses1,
                    'segment2': transformed_segment2,
                },
                "starting_eef_pose": starting_eef_pose_arm1 (list): [(4, 4)]
            },
            "arm2": {
                "overall_trajectory": overall_trajectory_arm2 (dict): {
                    'interp_to_segment1': interp_poses0,
                    'segment1': transformed_segment1,
                    'interp_to_segment2': interp_poses1,
                    'segment2': transformed_segment2,
                },
                "starting_eef_pose": starting_eef_pose_arm2 (list): [(4, 4)]
            }
        }
    '''

    arm1_segment1 = segment1[:, 0, :, :]
    arm1_segment2 = segment2[:, 0, :, :]
    arm2_segment1 = segment1[:, 1, :, :]
    arm2_segment2 = segment2[:, 1, :, :]
    starting_eef_pose_arm1 = starting_eef_pose[0] # [starting_eef_pose[0][0], starting_eef_pose[1][0]]
    starting_eef_pose_arm2 = starting_eef_pose[1] # [starting_eef_pose[0][1], starting_eef_pose[1][1]]

    overall_trajectory_arm1 = transform_two_segments_for_one_arm(
        segment1=arm1_segment1,
        segment2=arm1_segment2,
        src_obj1_pose=src_obj1_pose,
        cur_obj1_pose=cur_obj1_pose,
        src_obj2_pose=src_obj2_pose,
        cur_obj2_pose=cur_obj2_pose,
        starting_eef_pose=starting_eef_pose_arm1,
    )

    overall_trajectory_arm2 = transform_two_segments_for_one_arm(
        segment1=arm2_segment1,
        segment2=arm2_segment2,
        src_obj1_pose=src_obj1_pose,
        cur_obj1_pose=cur_obj1_pose,
        src_obj2_pose=src_obj2_pose,
        cur_obj2_pose=cur_obj2_pose,
        starting_eef_pose=starting_eef_pose_arm2,
    )

    return {
        "arm1": {
            "overall_trajectory": overall_trajectory_arm1,
            "starting_eef_pose": starting_eef_pose_arm1
        },
        "arm2": {
            "overall_trajectory": overall_trajectory_arm2,
            "starting_eef_pose": starting_eef_pose_arm2
        }
    }


In [12]:
cur_eef_poses

array([[[ 0.99324838,  0.01639563,  0.11484268, -0.11518818],
        [ 0.01569701, -0.99985254,  0.00698506, -0.2359546 ],
        [ 0.11494025, -0.00513521, -0.99335928,  1.10092079],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.99263872, -0.01135121,  0.12057997, -0.11328994],
        [-0.0120655 , -0.99991379,  0.00519528,  0.2468733 ],
        [ 0.12051059, -0.00661189, -0.9926901 ,  1.10284632],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]])

In [13]:
from vishal_dev.utils import segment_trajectory

segments = segment_trajectory(data, demo_id)

src_arm_segment1 = target_eef_poses[:segments[0][-1]+1, :]
src_arm_segment2 = target_eef_poses[:segments[1][-1]+1, :]

full_trajectory_outs = get_transformed_segments_for_both_arms(
    segment1=src_arm_segment1,
    segment2=src_arm_segment2,
    src_obj1_pose=src_lid_pose,
    cur_obj1_pose=new_lid_pose,
    src_obj2_pose=src_box_pose,
    cur_obj2_pose=new_box_pose,
    starting_eef_pose=cur_eef_poses,
)

<HDF5 dataset "lid_off_ground": shape (238,), type "<i8">


In [14]:
# transformed_data = get_transformed_segments_for_both_arms(
#     segment1=eef_poses[0],
#     segment2=eef_poses[1],
#     src_obj1_pose=src_lid_pose,
#     cur_obj1_pose=new_lid_pose,
#     src_obj2_pose=src_box_pose,
#     cur_obj2_pose=new_box_pose,
#     starting_eef_pose=target_eef_poses,
# )

In [15]:
'''
Execution order/logic for trajectories:

# 1. Interpolate to initial poses for each arm in parallel
N, M = len(arm1_interp_toseg1), len(arm2_interp_toseg1)

for i in range(max(N, M)):
    initialize action with zeros
    if i < N:
        set action[0:12] to arm1_interp_toseg1[i]
    if i < M:
        set action[12:24] to arm2_interp_toseg1[i]
    env.step(action)

# 2. Execute segment1 for each arm in parallel. This segment will be of same length for both arms

for i in range(len(arm1_segment1)):
    initialize action with zeros
    set action[0:12] to arm1_segment1[i]
    set action[12:24] to arm2_segment1[i]
    env.step(action)

# 3. Interpolate to initial poses of segment 2 for each arm in parallel
N, M = len(arm1_interp_toseg2), len(arm2_interp_toseg2)

for i in range(max(N, M)):
    initialize action with zeros
    if i < N:
        set action[0:12] to arm1_interp_toseg2[i]
    if i < M:
        set action[12:24] to arm2_interp_toseg2[i]
    env.step(action)

# 4. Execute segment2 for each arm in parallel. This segment will be of same length for both arms

for i in range(len(arm1_segment2)):
    initialize action with zeros
    set action[0:12] to arm1_segment2[i]
    set action[12:24] to arm2_segment2[i]
    env.step(action)

'''

'\nExecution order/logic for trajectories:\n\n# 1. Interpolate to initial poses for each arm in parallel\nN, M = len(arm1_interp_toseg1), len(arm2_interp_toseg1)\n\nfor i in range(max(N, M)):\n    initialize action with zeros\n    if i < N:\n        set action[0:12] to arm1_interp_toseg1[i]\n    if i < M:\n        set action[12:24] to arm2_interp_toseg1[i]\n    env.step(action)\n\n# 2. Execute segment1 for each arm in parallel. This segment will be of same length for both arms\n\nfor i in range(len(arm1_segment1)):\n    initialize action with zeros\n    set action[0:12] to arm1_segment1[i]\n    set action[12:24] to arm2_segment1[i]\n    env.step(action)\n\n# 3. Interpolate to initial poses of segment 2 for each arm in parallel\nN, M = len(arm1_interp_toseg2), len(arm2_interp_toseg2)\n\nfor i in range(max(N, M)):\n    initialize action with zeros\n    if i < N:\n        set action[0:12] to arm1_interp_toseg2[i]\n    if i < M:\n        set action[12:24] to arm2_interp_toseg2[i]\n    env.

In [16]:
'''
Reset environment to new starting poses
'''



'\nReset environment to new starting poses\n'

In [17]:
'''
Generate data

'''
interp_arm1_toseg1 = full_trajectory_outs["arm1"]["overall_trajectory"]["interp_to_segment1"]
interp_arm1_toseg2 = full_trajectory_outs["arm1"]["overall_trajectory"]["interp_to_segment2"]
interp_arm2_toseg1 = full_trajectory_outs["arm2"]["overall_trajectory"]["interp_to_segment1"]
interp_arm2_toseg2 = full_trajectory_outs["arm2"]["overall_trajectory"]["interp_to_segment2"]
seg1_arm1 = full_trajectory_outs["arm1"]["overall_trajectory"]["segment1"]
seg1_arm2 = full_trajectory_outs["arm2"]["overall_trajectory"]["segment1"]
seg2_arm1 = full_trajectory_outs["arm1"]["overall_trajectory"]["segment2"]
seg2_arm2 = full_trajectory_outs["arm2"]["overall_trajectory"]["segment2"]

In [24]:
PoseUtils.unmake_pose(interp_arm1_toseg1[i])[0] #, 0]

array([-0.11518818, -0.2359546 ,  1.10092079])

In [41]:
def eef_to_action_single_arm(eef_pos, eef_rot_mats, original_action, prev_eef_pos, prev_eef_quat, max_dpos=0.05, max_drot=0.5, cur_rot_from_env=False):
    """
    Convert end-effector pose target actions to simulation-executable action space.

    args:
        eef_pos (np.ndarray): target end-effector position (3,)
        eef_rot_mats (np.ndarray): target end-effector orientation as rotation matrices (3, 3)
        original_action (np.ndarray): original action to copy other dimensions from (15,)
        prev_eef_pos (np.ndarray): previous end-effector position (3,)
        prev_eef_quat (np.ndarray): previous end-effector orientation as quaternion (4,)
        max_dpos (float): maximum position delta
        max_drot (float): maximum rotation delta in axis-angle representation
        cur_rot_from_env (bool): rotation matrices from environment are slightly different from that ones from the dataset.
            If true, these rotation matrices must be adjusted before computing the delta rotation. Otherwise, the delta rotation
            will be incorrect.
    """
    max_dpos = 0.05
    max_drot = 0.5

    action = np.copy(original_action)

    print(eef_pos.shape, eef_rot_mats.shape, prev_eef_pos.shape, prev_eef_quat.shape, action.shape)
    

    # Set robot action in position space
    action[0:3] = eef_pos - prev_eef_pos #[0]
    action[0:3] = np.clip(action[0:3] / max_dpos, -1., 1.)

    # Set robot action in rotation space
    curr_rot = T.quat2mat(prev_eef_quat)
    target_rot = eef_rot_mats#[0] # T.quat2mat(prev_eef_quat[0])
    if cur_rot_from_env:
        curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
        curr_rot = -1 * curr_rot
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    action[3:6] = delta_rotation  
    return action

In [43]:
'''
Execute interpolation to initial poses for each arm in parallel
'''

# obs = obfull[0]
# reset_to(env, initial_state) # Reset this to some other state
# reset_to_state(env, states_trajectory[0])
# obfull = just_reset(env, initial_state) #  --- IGNORE ---

obs = obfull[0].copy()

N, M = len(interp_arm1_toseg1), len(interp_arm2_toseg1)

prev_action = action_trajectory[0]

for i in range(max(N, M)):

    action = prev_action.copy()

    if i == 0:
        if i < N:
            action[0:12] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm1_toseg1[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm1_toseg1[i])[1],
                original_action=action[0:12], # action_trajectory[0],
                prev_eef_pos=obs['robot0_eef_pos'],
                prev_eef_quat=obs['robot0_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )
        if i < M:
            action[12:24] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm2_toseg1[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm2_toseg1[i])[1],
                original_action=action[12:24], # action_trajectory[0],
                prev_eef_pos=obs['robot1_eef_pos'],
                prev_eef_quat=obs['robot1_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )

    else:   
        if i < N:
            action[0:12] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm1_toseg1[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm1_toseg1[i])[1],
                original_action=action[0:12], # action_trajectory[0],
                prev_eef_pos=obs['robot0_eef_pos'],
                prev_eef_quat=obs['robot0_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=True
            )   
        if i < M:
            action[12:24] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm2_toseg1[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm2_toseg1[i])[1],
                original_action=action[12:24], # action_trajectory[0],
                prev_eef_pos=obs['robot1_eef_pos'],
                prev_eef_quat=obs['robot1_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=True
            )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)


(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)


In [45]:
PoseUtils.unmake_pose(seg1_arm1[i])

(array([-0.14316011, -0.24456583,  1.09321805]),
 array([[-0.99597645, -0.08945754, -0.00531572],
        [-0.00507369, -0.00293271,  0.99998283],
        [-0.08947159,  0.99598632,  0.00246703]]))

In [46]:
'''
Execute transformed synced trajectory for each arm in parallel - segment 1

obs carried forward from previous loop
'''

# Assert both segments are of same length
assert len(seg1_arm1) == len(seg1_arm2)

N = len(seg1_arm1)

prev_action = action_trajectory[0]

for i in range(N):
    action = prev_action.copy()

    if i == 0:
        action[0:12] = eef_to_action_single_arm(
            eef_pos=PoseUtils.unmake_pose(seg1_arm1[i])[0],
            eef_rot_mats=PoseUtils.unmake_pose(seg1_arm1[i])[1],
            original_action=action_trajectory[i, 0:12], # action_trajectory[0],
            prev_eef_pos=obs['robot0_eef_pos'],
            prev_eef_quat=obs['robot0_eef_quat'], 
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=False, #cTrue
        )
    
        action[12:24] = eef_to_action_single_arm(
            eef_pos=PoseUtils.unmake_pose(seg1_arm2[i])[0],
            eef_rot_mats=PoseUtils.unmake_pose(seg1_arm2[i])[1],
            original_action=action_trajectory[i, 12:24], # action_trajectory[0],
            prev_eef_pos=obs['robot1_eef_pos'],
            prev_eef_quat=obs['robot1_eef_quat'],
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=False, #cTrue
        )

    else:   
        action[0:12] = eef_to_action_single_arm(
            eef_pos=PoseUtils.unmake_pose(seg1_arm1[i])[0],
            eef_rot_mats=PoseUtils.unmake_pose(seg1_arm1[i])[1],
            original_action=action_trajectory[i, 0:12], # action_trajectory[0],
            prev_eef_pos=obs['robot0_eef_pos'],
            prev_eef_quat=obs['robot0_eef_quat'],
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True
        )   
    
        action[12:24] = eef_to_action_single_arm(
            eef_pos=PoseUtils.unmake_pose(seg1_arm2[i])[0],
            eef_rot_mats=PoseUtils.unmake_pose(seg1_arm2[i])[1],
            original_action=action_trajectory[i, 12:24], # action_trajectory[0],
            prev_eef_pos=obs['robot1_eef_pos'],
            prev_eef_quat=obs['robot1_eef_quat'],
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True
        )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)


(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,

In [48]:
'''
Execute interpolation to initial poses of segment 2 for each arm in parallel
'''

# obs = obfull[0]
# reset_to(env, initial_state) # Reset this to some other state
# reset_to_state(env, states_trajectory[0])
obfull = just_reset(env, initial_state) #  --- IGNORE ---

obs = obfull[0].copy()

N, M = len(interp_arm1_toseg2), len(interp_arm2_toseg2)

# prev_action = action_trajectory[0]

for i in range(max(N, M)):

    action = prev_action.copy()

    if i == 0:
        if i < N:
            action[0:12] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm1_toseg2[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm1_toseg2[i])[1],
                original_action=prev_action[0:12], # action_trajectory[0],
                prev_eef_pos=obs['robot0_eef_pos'],
                prev_eef_quat=obs['robot0_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )
        if i < M:
            action[12:24] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm2_toseg2[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm2_toseg2[i])[1],
                original_action=prev_action[12:24], # action_trajectory[0],
                prev_eef_pos=obs['robot1_eef_pos'],
                prev_eef_quat=obs['robot1_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )

    else:   
        if i < N:
            action[0:12] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm1_toseg2[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm1_toseg2[i])[1],
                original_action=prev_action[0:12], # action_trajectory[0],
                prev_eef_pos=obs['robot0_eef_pos'],
                prev_eef_quat=obs['robot0_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=True
            )   
        if i < M:
            action[12:24] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm2_toseg2[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm2_toseg2[i])[1],
                original_action=prev_action[12:24], # action_trajectory[0],
                prev_eef_pos=obs['robot1_eef_pos'],
                prev_eef_quat=obs['robot1_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=True
            )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)


(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)


In [ ]:
'''
Execute transformed synced trajectory for each arm in parallel - segment 2

obs carried forward from previous loop
'''

# Assert both segments are of same length
assert len(seg2_arm1) == len(seg2_arm2)

N = len(seg2_arm1)

prev_action = action_trajectory[0]

for i in range(N):
    action = prev_action.copy()

    if i == 0:
        action[0:12] = eef_to_action_single_arm(
            eef_pos=PoseUtils.unmake_pose(seg2_arm1[i])[0],
            eef_rot_mats=PoseUtils.unmake_pose(seg2_arm1[i])[1],
            original_action=action_trajectory[i+len(segments[0])][0:12], # action_trajectory[0],
            prev_eef_pos=obs['robot0_eef_pos'],
            prev_eef_quat=obs['robot0_eef_quat'],
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=False, #cTrue
        )
    
        action[12:24] = eef_to_action_single_arm(
            eef_pos=PoseUtils.unmake_pose(seg2_arm2[i])[0],
            eef_rot_mats=PoseUtils.unmake_pose(seg2_arm2[i])[1],
            original_action=action_trajectory[i+len(segments[0])][12:24], # action_trajectory[0],
            prev_eef_pos=obs['robot1_eef_pos'],
            prev_eef_quat=obs['robot1_eef_quat'],
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=False, #cTrue
        )

    else:   
        action[0:12] = eef_to_action_single_arm(
            eef_pos=PoseUtils.unmake_pose(seg2_arm1[i])[0],
            eef_rot_mats=PoseUtils.unmake_pose(seg2_arm1[i])[1],
            original_action=action_trajectory[i+len(segments[0])][0:12], # action_trajectory[0],
            prev_eef_pos=obs['robot0_eef_pos'],
            prev_eef_quat=obs['robot0_eef_quat'],
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True
        )   
    
        action[12:24] = eef_to_action_single_arm(
            eef_pos=PoseUtils.unmake_pose(seg2_arm2[i])[0],
            eef_rot_mats=PoseUtils.unmake_pose(seg2_arm2[i])[1],
            original_action=action_trajectory[i+len(segments[0])][12:24], # action_trajectory[0],
            prev_eef_pos=obs['robot1_eef_pos'],
            prev_eef_quat=obs['robot1_eef_quat'],
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True
        )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)


(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,

IndexError: Index (238) out of range for (0-237)